# Worker Safety Detection System - Testing Notebook

This notebook provides interactive examples for testing the worker safety detection system.

## Contents
1. Setup and Configuration
2. Load YOLOv8 Model
3. Test on Sample Images
4. Test Telegram Alerts
5. Process Sample Video
6. Visualize Results

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Add src directory to path
sys.path.append('../src')

import cv2
import yaml
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from IPython.display import Image, display

# For Telegram testing
import asyncio
from alert import test_telegram_connection, send_violation_alert
from utils import draw_boxes, save_violation_screenshot

print("✅ All imports successful!")

In [ ]:
# Load configuration
config_path = '../config/config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"Model path: {config['detection']['model_path']}")
print(f"Confidence threshold: {config['detection']['confidence_threshold']}")
print(f"Telegram alerts enabled: {config['telegram']['enable_alerts']}")

## 2. Load YOLOv8 Model

In [ ]:
# Load YOLOv8 model
model_path = f"../{config['detection']['model_path']}"

print(f"Loading model from {model_path}...")
model = YOLO(model_path)
print("✅ Model loaded successfully!")

# Display model info
print(f"\nModel classes: {model.names}")

## 3. Test on Sample Images

In [ ]:
# Helper function to display images in notebook
def show_image(img, title="Image", figsize=(12, 8)):
    """Display image in notebook"""
    plt.figure(figsize=figsize)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Helper function for detection
def detect_and_visualize(image_path, model, config):
    """Run detection and visualize results"""
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Could not read image: {image_path}")
        return
    
    print(f"Image shape: {img.shape}")
    
    # Run detection
    results = model(img, conf=config['detection']['confidence_threshold'])
    
    # Display results
    result = results[0]
    print(f"\nDetections: {len(result.boxes)} objects found")
    
    # Print detected classes
    for box in result.boxes:
        class_id = int(box.cls[0])
        class_name = result.names[class_id]
        confidence = float(box.conf[0])
        print(f"  - {class_name}: {confidence:.2f}")
    
    # Visualize
    annotated = result.plot()
    show_image(annotated, f"Detections: {image_path}")
    
    return results

print("Helper functions defined!")

In [ ]:
# Example: Test on an image
# Replace with your own image path
test_image = "../data/images/test_image.jpg"

# Check if image exists
if os.path.exists(test_image):
    results = detect_and_visualize(test_image, model, config)
else:
    print(f"❌ Test image not found: {test_image}")
    print("Place a test image in data/images/ folder and update the path above.")

## 4. Test Telegram Alerts

In [ ]:
# Test Telegram connection
bot_token = config['telegram']['bot_token']
chat_id = config['telegram']['chat_id']

if bot_token != "YOUR_BOT_TOKEN_HERE" and chat_id != "YOUR_CHAT_ID_HERE":
    print("Testing Telegram connection...")
    await test_telegram_connection(bot_token, chat_id)
else:
    print("⚠️ Please configure Telegram bot token and chat ID in config/config.yaml")

In [ ]:
# Test sending a violation alert with sample data
if bot_token != "YOUR_BOT_TOKEN_HERE" and chat_id != "YOUR_CHAT_ID_HERE":
    # Create sample violation data
    sample_violations = [
        {
            'class': 'no-helmet',
            'confidence': 0.89,
            'bbox': [100, 150, 300, 400]
        },
        {
            'class': 'no-safety-vest',
            'confidence': 0.76,
            'bbox': [120, 200, 280, 450]
        }
    ]
    
    # You would need an actual image file for this to work
    # sample_image_path = "path/to/sample/image.jpg"
    # await send_violation_alert(bot_token, chat_id, sample_image_path, sample_violations)
    
    print("To test alerts, uncomment the code above and provide a valid image path.")
else:
    print("⚠️ Please configure Telegram credentials first.")

## 5. Process Sample Video

In [ ]:
# Process a video file
from detect import SafetyDetector

video_path = "../data/videos/test.mp4"

if os.path.exists(video_path):
    print(f"Processing video: {video_path}")
    
    # Initialize detector
    detector = SafetyDetector('../config/config.yaml')
    
    # Process video
    detector.process_video(video_path)
    
    print("\n✅ Video processing complete!")
    print("Check output/videos/ for the annotated video.")
    print("Check output/violations/ for violation screenshots.")
else:
    print(f"❌ Video not found: {video_path}")
    print("Place a test video in data/videos/ folder.")

## 6. Visualize Detection Results

In [ ]:
# Visualize some frames from the processed video
def visualize_video_frames(video_path, num_frames=5):
    """Extract and visualize random frames from video"""
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"❌ Could not open video: {video_path}")
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    
    # Select random frames
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)
    
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        
        if ret:
            # Run detection
            results = model(frame, conf=config['detection']['confidence_threshold'])
            annotated = results[0].plot()
            
            show_image(annotated, f"Frame {idx}/{total_frames}")
    
    cap.release()
    print("✅ Visualization complete!")

# Uncomment to visualize frames from your video
# visualize_video_frames("../data/videos/test.mp4", num_frames=3)

## 7. View Saved Violation Screenshots

In [ ]:
# List and display violation screenshots
violations_dir = "../output/violations/"

if os.path.exists(violations_dir):
    violation_images = list(Path(violations_dir).glob("*.jpg"))
    
    print(f"Found {len(violation_images)} violation screenshots")
    
    # Display first few violations
    for img_path in violation_images[:5]:  # Show first 5
        img = cv2.imread(str(img_path))
        if img is not None:
            show_image(img, f"Violation: {img_path.name}")
else:
    print("No violations found yet. Run video processing first.")

## 8. Statistics and Analysis

In [ ]:
# Analyze violation patterns
violations_dir = Path("../output/violations/")

if violations_dir.exists():
    violation_files = list(violations_dir.glob("*.jpg"))
    
    print(f"📊 Violation Statistics")
    print(f"="*50)
    print(f"Total violation screenshots: {len(violation_files)}")
    
    if violation_files:
        # Get file timestamps
        timestamps = [f.stem.split('_')[1:] for f in violation_files]
        print(f"First violation: {violation_files[0].name}")
        print(f"Latest violation: {violation_files[-1].name}")
else:
    print("No violations directory found.")

## Next Steps

1. **Train Custom Model**: For better accuracy, train YOLOv8 on a PPE-specific dataset
2. **Live Camera**: Integrate with RTSP streams for real-time monitoring
3. **Dashboard**: Create a web dashboard for monitoring multiple cameras
4. **Database**: Store violation records in a database for analytics
5. **Multi-camera**: Extend to monitor multiple cameras simultaneously

---

**Happy Testing! 🚀**